## Assignment 08

#### Submitted By:
1. Dhruvan Ganesh
2. Sheikh Mastura Farzana

In [1]:
%load_ext tensorboard

### Environment Information
---

In [2]:
import torch.cuda
from pytorch_lightning import seed_everything
from torch import device

device = device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)
if torch.cuda.is_available():
    print("Device Name:", torch.cuda.get_device_name(0))

seed = 42
seed_everything(seed)
print("Set Random Seed:", seed)

data_dir = "~/.datasets"
print("Data Dir:", data_dir)

Device: cuda:0
Device Name: GeForce GTX 1660 Ti
Set Random Seed: 42
Data Dir: ~/.datasets


# Assignment8 (21 Aug 2020)
---
- Train a DCGAN for humanoid robot
- Extra point for using WGAN

## <span style="font-variant:small-caps">Task 1: Train a DCGAN for Humanoid Robot</span>
---

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def get_kwargs(**kwargs):
    return kwargs

def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation="nearest")
    plt.show()

In [4]:
import torch
from pytorch_lightning import LightningModule

import torch.nn as nn

In [5]:
# G(z)
class Generator(nn.Module):
    # initializers
    def __init__(self, d=128, color_channels=1, latent_dim=100):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, d * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(d * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(d * 8, d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(d * 4, d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(d * 2, d, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d),
            nn.ReLU(True),
            nn.ConvTranspose2d(d, color_channels, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

        self.weight_init(mean=0.0, std=0.02)

    # weight_init
    def weight_init(self, mean, std):
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)

    # forward method
    def forward(self, x):
        x = self.net(x)

        return x


class Discriminator(nn.Module):
    # initializers
    def __init__(self, d=128, color_channels=1):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(color_channels, d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d, d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d * 2, d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d * 4, d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(d * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
        )

        self.weight_init(mean=0.0, std=0.02)

    # weight_init
    def weight_init(self, mean, std):
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)

    # forward method
    def forward(self, x):
        x = self.net(x)
        return x.view(-1, 1).squeeze(1)

In [6]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader


class RobotModule(LightningModule):
    def __init__(self, trainset_path: str, batch_size: int, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.preprocess = transforms.Compose(
            [
                transforms.Resize(64),
                transforms.CenterCrop(64),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )
        self.undo_preprocess = transforms.Normalize(
            mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
            std=[1 / 0.229, 1 / 0.224, 1 / 0.225],
        )

        self.trainset = datasets.ImageFolder(
            root=trainset_path, transform=self.preprocess
        )
        self.classes = self.trainset.classes
        self.batch_size = batch_size
        self.color_channels = 3

    def train_dataloader(self):
        loader = DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            num_workers=4,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
        )
        return loader

In [7]:
from collections import OrderedDict
import torchvision


def make_grid(tensors=(), idxs=()):
    out_tensors = []
    for tensor in tensors:
        for idx in idxs:
            out_tensors.append(tensor[idx])
    return out_tensors


class DCGAN(RobotModule):
    def __init__(self, batch_size, **kwargs):
        super().__init__(batch_size=batch_size, **kwargs)
        self.batch_size = batch_size
        self.latent_dim = 100

        self.generator = Generator(
            64, color_channels=self.color_channels, latent_dim=self.latent_dim
        )
        self.discriminator = Discriminator(64, color_channels=self.color_channels)

        # cache for generated images
        self.generated_imgs = None
        self.last_imgs = None

        self.adversarial_loss = nn.BCELoss()

    def configure_optimizers(self):
        opt_g = torch.optim.Adam(
            self.generator.parameters(), lr=0.0002, betas=(0.5, 0.999)
        )
        opt_d = torch.optim.Adam(
            self.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999)
        )
        return [opt_g, opt_d], []

    def forward(self, x):
        x = self.generator(x)
        return x

    def training_step(self, batch, batch_idx, optimizer_idx):
        imgs, _ = batch

        # sample noise
        z = torch.randn(imgs.shape[0], self.latent_dim, 1, 1)
        z = z.type_as(imgs)

        # train generator
        if optimizer_idx == 0:

            # generate images
            self.generated_imgs = self(z)

            # log sampled images
            sample_imgs = self.generated_imgs[:16]
            grid = torchvision.utils.make_grid(sample_imgs, nrow=4, normalize=True)
            torchvision.utils.save_image(
                grid, f"/tmp/gan-gen-img-{self.current_epoch}.jpg"
            )
            self.logger.experiment.add_image("generated_images", grid, 0)

            # ground truth result (ie: all fake)
            # put on GPU because we created this tensor inside training_loop
            valid = torch.ones(imgs.size(0), 1)
            valid = valid.type_as(imgs)

            # adversarial loss is binary cross-entropy
            g_loss = self.adversarial_loss(self.discriminator(self(z)), valid)
            tqdm_dict = {"loss_g": g_loss}
            output = OrderedDict(
                {"loss": g_loss, "progress_bar": tqdm_dict, "log": tqdm_dict}
            )
            return output

        # train discriminator
        if optimizer_idx == 1:
            # Measure discriminator's ability to classify real from generated samples

            # how well can it label as real?
            valid = torch.ones(imgs.size(0), 1)
            valid = valid.type_as(imgs)

            real_loss = self.adversarial_loss(self.discriminator(imgs), valid)

            # how well can it label as fake?
            fake = torch.zeros(imgs.size(0), 1)
            fake = fake.type_as(imgs)

            fake_loss = self.adversarial_loss(
                self.discriminator(self(z).detach()), fake
            )

            # discriminator loss is the average of these
            d_loss = (real_loss + fake_loss) / 2
            tqdm_dict = {"loss_d": d_loss}
            output = OrderedDict(
                {"loss": d_loss, "progress_bar": tqdm_dict, "log": tqdm_dict}
            )
            return output

In [8]:
%tensorboard --logdir lightning_logs

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

   | Name                 | Type            | Params
-----------------------------------------------------
0  | generator            | Generator       | 3 M   
1  | generator.net        | Sequential      | 3 M   
2  | generator.net.0      | ConvTranspose2d | 819 K 
3  | generator.net.1      | BatchNorm2d     | 1 K   
4  | generator.net.2      | ReLU            | 0     
5  | generator.net.3      | ConvTranspose2d | 2 M   
6  | generator.net.4      | BatchNorm2d     | 512   
7  | generator.net.5      | ReLU            | 0     
8  | generator.net.6      | ConvTranspose2d | 524 K 
9  | generator.net.7      | BatchNorm2d     | 256   
10 | generator.net.8      | ReLU            | 0     
11 | generator.net.9      | ConvTranspose2d | 131 K 
12 | generator.net.10     | BatchNorm2d     | 128   
13 | generator.net.11     | ReLU            | 0     
14 | generator.net.12     | ConvTra

1

In [ ]:
from pytorch_lightning import Trainer

gan_model = DCGAN(batch_size=32, trainset_path="minimal_dataset/")

trainer = Trainer(gpus=1, max_epochs=200, checkpoint_callback=False,)
trainer.fit(gan_model)